In [ ]:
## IDEAS:

## Are nfl games worse now?

# 1.   One score games
    # Pregame spread
    # Score differential =<7
    # Score/ Drive!!!, PPG

# 2. Penalties!!!

# 3.  dominant games by skill players 
    # How to define "dominant game"?
    # Use fantasy points against standard deviation of season average
# Latest lead change

# In general, find average first
    # then tier? Standard deviation


In [81]:
## Bring in source, import game data

import nfl_data_py as nfl
import pandas as pd
import numpy as np

years = range(2000,2024)

games_df = nfl.import_schedules(years)


In [82]:
cols = ["game_id", "old_game_id", "week", "penalty"]

pbp = nfl.import_pbp_data([2023,2024], columns = cols)

2023 done.
2024 done.
Downcasting floats.


In [83]:
# Find score difference

games_df['score_diff'] = abs( games_df['home_score'] - games_df['away_score'])

# Create boolean column for one score games (>= 8 for TD w/ 2 pt)

games_df['one_score'] = (games_df['score_diff'] <= 8).astype(int)

In [84]:
# Find averages by season: one_score, ppg

seasons_df = (
    games_df
    .groupby('season')
    .agg(
        avg_total = ('total', 'mean'),
        one_scores = ('one_score', 'sum')
    )
    .reset_index()
    .round(2)
)


In [85]:
# Import player weekly data FF scoring, determine averages and benchmarks per season
# cols: 'fantasy_points', 'fantasy_points_ppr'

#import data, filter variable

stats_df = nfl.import_weekly_data(years)

positions_to_keep = ['QB', 'RB', 'WR', 'TE']

# Rank player total 



avg_stats_df = (
    stats_df[stats_df['position'].isin(positions_to_keep)]
    .groupby(['season', 'position'])
    .agg(
        avg_fpts = ('fantasy_points', np.mean),
        dominant_games = ('fantasy_points', lambda x: (x >25).sum()))
    .reset_index()
)



Downcasting floats.


In [88]:
# Merge dfs on season

player_team_season_df = avg_stats_df.merge(seasons_df, how='left', on='season')

print(player_team_season_df)

    season position   avg_fpts  dominant_games  avg_total  one_scores
0     2000       QB  10.410015              45      41.09         121
1     2000       RB   7.270330              49      41.09         121
2     2000       TE   2.982612               0      41.09         121
3     2000       WR   5.496795              19      41.09         121
4     2001       QB  11.151531              30      40.54         132
..     ...      ...        ...             ...        ...         ...
91    2022       WR   5.297221              19      44.03         164
92    2023       QB  13.364290              68      43.80         154
93    2023       RB   6.791903              25      43.80         154
94    2023       TE   3.666595               1      43.80         154
95    2023       WR   5.276641              21      43.80         154

[96 rows x 6 columns]


In [90]:
player_team_season_df.to_csv("/Users/michaelrozewski/Desktop/fantasy prject/bad_games/player_team_season.csv")